In [123]:
import numpy as np
import pandas as pd
import os
import math
import pyrr
from datetime import datetime, timedelta
from sklearn import linear_model

In [139]:
data_directory = "../Data/"
hololenses = ['A/', 'B/']
intermediate = 'static/'
subdirectory = "shifted/"

In [105]:
def arctan(y, x):
    result = math.degrees(math.atan2(y, x))
    if x<0 and y<0:
        return result + 180
    elif x<0 and y>0:
        return result + 180
    elif x>0 and y<0:
        return result + 360
    else:
        return result

In [126]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    result['globalTargetVector.x'] = result['transform.position.x']-result['HeadGazeOrigin.x']
    result['globalTargetVector.y'] = result['transform.position.y']-result['HeadGazeOrigin.y']
    result['globalTargetVector.z'] = result['transform.position.z']-result['HeadGazeOrigin.z']
    result['localEyeGazeX'] = result.apply(lambda row: pyrr.quaternion.apply_to_vector(pyrr.quaternion.inverse(pyrr.quaternion.create(x=row['HeadRotation.x'], y=row['HeadRotation.y'], z=row['HeadRotation.z'], w=row['HeadRotation.w'])), np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])))[0], axis=1)
    result['localEyeGazeY'] = result.apply(lambda row: pyrr.quaternion.apply_to_vector(pyrr.quaternion.inverse(pyrr.quaternion.create(x=row['HeadRotation.x'], y=row['HeadRotation.y'], z=row['HeadRotation.z'], w=row['HeadRotation.w'])), np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])))[1], axis=1)
    result['localEyeGazeZ'] = result.apply(lambda row: pyrr.quaternion.apply_to_vector(pyrr.quaternion.inverse(pyrr.quaternion.create(x=row['HeadRotation.x'], y=row['HeadRotation.y'], z=row['HeadRotation.z'], w=row['HeadRotation.w'])), np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])))[2], axis=1)
    # result['GazeAngleX'] = result.apply(lambda row: math.degrees(math.atan2(row['localEyeGazeX'], row['localEyeGazeZ'])), axis=1)
    # result['GazeAngleY'] = result.apply(lambda row: math.degrees(math.atan2(row['localEyeGazeY'], row['localEyeGazeZ'])), axis=1)
    # result['TargetAngleX'] = result.apply(lambda row: math.degrees(math.atan2(row['localTransform.position.x'], row['localTransform.position.z'])), axis=1)
    # result['TargetAngleY'] = result.apply(lambda row: math.degrees(math.atan2(row['localTransform.position.y'], row['localTransform.position.z'])), axis=1)
    result['GazeAngleX'] = np.degrees(np.arctan2(result['localEyeGazeX'], result['localEyeGazeZ']))
    result['GazeAngleY'] = np.degrees(np.arctan2(result['localEyeGazeY'], result['localEyeGazeZ']))
    result['TargetAngleX'] = np.degrees(np.arctan2(result['localTransform.position.x'], result['localTransform.position.z']))
    result['TargetAngleY'] = np.degrees(np.arctan2(result['localTransform.position.y'], result['localTransform.position.z']))
    result.reset_index(inplace=True, drop=True)
    return result

In [127]:
def create_calibration_df(taskName, fileList, participant, hololens):
    frames = []
    for file in fileList:
        if taskName in file:
            filePath = data_directory + hololens + participant + '/' + intermediate + subdirectory + file
            df = pd.read_csv(filePath)
            df = clean_df(df)
            frames.append(df)
    if len(frames) == 0:
        print("error: task not found")
        return None
    else:
        result = pd.concat(frames)
        return result

In [48]:
def calculate_regression_coefficients(df):
    x1 = df[['GazeAngleX','GazeAngleY']]
    y1 = df['TargetAngleX']
    x2 = df[['GazeAngleX','GazeAngleY']]
    y2 = df['TargetAngleY']
    x_regr = linear_model.LinearRegression()
    x_regr.fit(x1.to_numpy(), y1.to_numpy())
    y_regr = linear_model.LinearRegression()
    y_regr.fit(x2.to_numpy(), y2.to_numpy())
    return [x_regr, y_regr]

In [122]:
def calculate_cosine_error(df):
    #df['cosineErrorGlobal'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z'])), np.array((row['globalTargetVector.x'], row['globalTargetVector.y'], row['globalTargetVector.z'])))/(np.linalg.norm(np.array((row['EyeGazeDirection.x'], row['EyeGazeDirection.y'], row['EyeGazeDirection.z']))) * np.linalg.norm(np.array((row['globalTargetVector.x'], row['globalTargetVector.y'], row['globalTargetVector.z'])))))), axis=1)
    df['cosineError'] = df.apply(lambda row: np.degrees(np.arccos(np.dot(np.array((row['localEyeGazeX'], row['localEyeGazeY'], row['localEyeGazeZ'])), np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))/(np.linalg.norm(np.array((row['localEyeGazeX'], row['localEyeGazeY'], row['localEyeGazeZ']))) * np.linalg.norm(np.array((row['localTransform.position.x'], row['localTransform.position.y'], row['localTransform.position.z'])))))), axis=1)
    return df

In [49]:
def calculate_euclidean_error(df):
    #df['euclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['GazeAngleX'], row['GazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    #df['recalibratedEuclideanError'] = df.apply(lambda row: np.linalg.norm(np.array((row['CalibratedGazeAngleX'], row['CalibratedGazeAngleY']))-np.array((row['TargetAngleX'], row['TargetAngleY']))), axis=1)
    df['euclideanError'] = np.sqrt((df.GazeAngleX-df.TargetAngleX) * (df.GazeAngleX-df.TargetAngleX) + (df.GazeAngleY-df.TargetAngleY) * (df.GazeAngleY-df.TargetAngleY))
    df['recalibratedEuclideanError'] = np.sqrt((df.CalibratedGazeAngleX-df.TargetAngleX) * (df.CalibratedGazeAngleX-df.TargetAngleX) + (df.CalibratedGazeAngleY-df.TargetAngleY) * (df.CalibratedGazeAngleY-df.TargetAngleY))
    return df

In [128]:
def calibrate_files(taskname):
    output_subdirectory = intermediate + "recalibrated_" + taskname
    for hololens in hololenses:
        participantList = os.listdir(data_directory + hololens)
        for participant in participantList:
            print(participant)
            if participant == '.DS_Store':
                print("skipping calibration")
                continue
            if os.path.exists(data_directory + hololens + participant + '/' + output_subdirectory) and len(os.listdir(data_directory + hololens + participant + '/' + output_subdirectory)) == 12:
                print("already calibrated")
                continue
            fileList = os.listdir(data_directory + hololens + participant + '/' + intermediate + subdirectory)
            calibration_df = create_calibration_df(taskname, fileList, participant, hololens)
            [x_coeff, y_coeff] = calculate_regression_coefficients(calibration_df)
            for file in fileList:
                filePath = data_directory + hololens + participant + '/' + intermediate + subdirectory + file
                if '.csv' in file:
                    df = pd.read_csv(filePath)
                    df = clean_df(df)
                    df['CalibratedGazeAngleX'] = df.apply(lambda row: x_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
                    df['CalibratedGazeAngleY'] = df.apply(lambda row: y_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
                    df = calculate_cosine_error(df)
                    df = calculate_euclidean_error(df)
                    recalibrated_df_output_directory = data_directory + hololens + participant + '/' + output_subdirectory
                    recalibrated_df_output_path = data_directory + hololens + participant + '/' + output_subdirectory + '/' + file
                    if not os.path.exists(recalibrated_df_output_directory):
                        os.mkdir(recalibrated_df_output_directory)
                    #filePrefix = file.split('.')[0]
                    #recalibrated_df_output_path = data_directory + output_subdirectory + '/' + filePrefix + '_recalibrated_' + taskname + '.csv'
                    df.to_csv(recalibrated_df_output_path, index=False)

In [140]:
calibrate_files('calibration')

03
04
32
34
05
02
.DS_Store
skipping calibration
18
27
29
10
19
43
07
36
09
53
37
08
01
06
39
52
46
41
49
47
25
50
35
51
33
.DS_Store
skipping calibration
20
11
16
42
45
28
17
26
21
44
38
31
54
30
48
24
23
15
12
40
13
14
22


In [130]:
tasknames = ['calibration', 'ssHeadConstrained', 'wsBodyConstrained', 'ssWalking', 'wsWalking', 'hallway']
for taskname in tasknames:
    print(taskname)
    calibrate_files(taskname)

calibration
03
already calibrated
04
already calibrated
32
already calibrated
34
already calibrated
05
already calibrated
02
already calibrated
.DS_Store
skipping calibration
18
already calibrated
27
already calibrated
29
already calibrated
10
already calibrated
19
already calibrated
43
already calibrated
07
already calibrated
36
already calibrated
09
already calibrated
53
already calibrated
37
already calibrated
08
already calibrated
01
already calibrated
06
already calibrated
39
already calibrated
52
already calibrated
46
already calibrated
41
already calibrated
49
already calibrated
47
already calibrated
25
already calibrated
50
already calibrated
35
already calibrated
51
already calibrated
33
already calibrated
.DS_Store
skipping calibration
20
already calibrated
11
already calibrated
16
already calibrated
42
already calibrated
45
already calibrated
28
already calibrated
17
already calibrated
26
already calibrated
21
already calibrated
44
already calibrated
38
already calibrated
31